In [1]:
import pandas as pd
import numpy as np
import requests
import re
import csv

In [2]:
POWHEG_NRes = { 'cHHH1'   :'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc700/13TeV/powheg/V2/ggHH_EWChL_NNPDF31_13TeV_M125_cHHH1/v3/ggHH_EWChL_slc6_amd64_gcc700_CMSSW_10_2_5_patch1_my_ggHH_EWChL.tgz',
                'cHHH0'   :'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc700/13TeV/powheg/V2/ggHH_EWChL_NNPDF31_13TeV_M125_cHHH0/v3/ggHH_EWChL_cHHH0_slc6_amd64_gcc700_CMSSW_10_2_5_patch1_my_ggHH_EWChL.tgz',
                'cHHH5'   :'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc700/13TeV/powheg/V2/ggHH_EWChL_NNPDF31_13TeV_M125_cHHH5/v3/ggHH_EWChL_cHHH5_slc6_amd64_gcc700_CMSSW_10_2_5_patch1_my_ggHH_EWChL.tgz',
                'cHHH2p45':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc700/13TeV/powheg/V2/ggHH_EWChL_NNPDF31_13TeV_M125_cHHH2p45/v3/ggHH_EWChL_cHHH2p45_slc6_amd64_gcc700_CMSSW_10_2_5_patch1_my_ggHH_EWChL.tgz'}

In [22]:
process_gridpacks_map = {
'GluGluToHHToPROCESS_node_VALUE_UETUNE_PSWeights_13TeV-powheg-pythia8' : POWHEG_NRes,
}

LHEproducer = """import FWCore.ParameterSet.Config as cms

externalLHEProducer = cms.EDProducer("ExternalLHEProducer",
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)

#Link to datacards:
#https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/exo_diboson/
"""

num_events = {
    '4B'         : 1000000,
    '2B2Tau'     : 1000000,
    '2B2VTo2L2Nu':  400000,
    '2B2G'       :  200000,
    '4Tau'       : 1000000,
    '2B2ZTo2L2J' :  400000,
    '2B2ZTo4L'   :  400000,
    '4V'         : 1000000,
    '2B2VLNu2J'  :  400000,
    '2V2Tau'     :  400000
}

In [6]:
import fragments

In [23]:
Tune_pre2017 = 'TuneCUETP8M1'
Tune_2017    = 'TuneCP5'

years = ['2016', '2017', '2018']

for year in years:

    with open('HH_NonRes_Powheg'+year+'.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',
                                quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['Dataset name','Events', 'fragment','notes','Generator'])
    
        for channel, nevents in num_events.items():
            fragment_dict = fragments.HHchannel_dict[channel]
               
            for key, value in process_gridpacks_map.items():

                for value, path in value.items():
                    #print('\t %s - %s' %(key.replace('VALUE',value),path))
                    dataset_name = key.replace('PROCESS',channel).replace('VALUE',value).replace('UETUNE', Tune_pre2017 if year == '2016' else Tune_2017)
                    events=nevents
                    final_fragment = LHEproducer.replace('__GRIDPACK__',path) + '\n' + fragment_dict[year]
                    note = dataset_name.replace('_',' ')
                    generators="Powheg Pythia8"
                    csvwriter.writerow([dataset_name, events, final_fragment, note, generators])

In [24]:
!cat HH_NonRes_Powheg2016.csv


GluGluToHHTo4B_node_cHHH1_TuneCUETP8M1_PSWeights_13TeV-powheg-pythia8,1000000,"import FWCore.ParameterSet.Config as cms

externalLHEProducer = cms.EDProducer(""ExternalLHEProducer"",
    args = cms.vstring('/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc700/13TeV/powheg/V2/ggHH_EWChL_NNPDF31_13TeV_M125_cHHH1/v3/ggHH_EWChL_slc6_amd64_gcc700_CMSSW_10_2_5_patch1_my_ggHH_EWChL.tgz'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)

#Link to datacards:
#https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/exo_diboson/

import FWCore.ParameterSet.Config as cms
from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.Pythia8CUEP8M1Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweights